In [39]:
import scipy
import netCDF4
import datetime
import cmocean
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cpf
from cartopy.feature import NaturalEarthFeature, LAND, COASTLINE

# Constants

In [40]:
sizefont = 20
sizefont_legend = 20
date_task = "20240421"
#
paths = {}
paths["AICE"] = "/lustre/storeB/project/fou/hi/oper/aice/archive/"
paths["output"] = "/lustre/storeB/users/cyrilp/AICE/Figures/Paper/"
#
xmin = 274
xmax = 364
ymin = 154
ymax = 244
#
lead_times = [0, 4, 9]
#
spatial_resolution = 5000
#
map_params = {"LAND_highres": cpf.NaturalEarthFeature("physical", "land", "50m", edgecolor = "face", facecolor = "tan", linewidth = 0.1),
              "map_extent": (15, 50, 78, 80),
              "map_proj": ccrs.LambertAzimuthalEqualArea(central_longitude = 0, central_latitude = 79, false_easting = 0.0, false_northing = 0.0, globe = None),
              "levels": np.linspace(0, 100, 21),
              "norm": BoundaryNorm(np.linspace(0, 100, 21), 256),
              "colormap": plt.cm.cubehelix}

# Read_data

In [41]:
def read_AICE(date_task, paths, xmin, xmax, ymin, ymax):
    Dataset = {}
    filename = paths["AICE"] + "AICE_forecasts_" + date_task + "T000000Z.nc"
    with netCDF4.Dataset(filename, "r") as nc:
        for var in nc.variables:
            if var == "x":
                Dataset[var] = nc.variables[var][xmin:xmax]
            elif var == "y":
                Dataset[var] = nc.variables[var][ymin:ymax]
            elif var == "lat" or var == "lon":
                Dataset[var] = nc.variables[var][ymin:ymax, xmin:xmax]
            elif var == "SIC":
                Dataset[var] = nc.variables[var][:, ymin:ymax, xmin:xmax]
                Dataset[var][np.isnan(Dataset[var])] = 0
    return Dataset

# Spectral variance

In [42]:
def spectral_variance(sstarr, dx):

    dctarr = scipy.fftpack.dct(scipy.fftpack.dct(sstarr, axis = 0, type = 2, norm = "ortho"), axis = 1, type = 2, norm = "ortho")

    Ni = np.shape(dctarr)[1]
    Nj = np.shape(dctarr)[0]

    Ni_inds, Nj_inds = np.meshgrid(np.arange(Ni), np.arange(Nj))

    # Total variance array:
    totvar = np.abs(dctarr)**2/(Ni*Nj) # each element on a given circle has the same wavenumber k

    vararr = np.zeros(min([Ni, Nj]))
    wavelengtharrmin = np.zeros(min([Ni, Nj]))
    wavelengtharrmax = np.zeros(min([Ni, Nj]))
    wavelengtharrmid = np.zeros(min([Ni, Nj]))

    for k in range(1, min([Ni, Nj])):
        # For a given k, determine the limits of the contributing band defined by alpha(k) and alpha(k) + delta(alpha(k)):
        alphamin = k/min([Ni, Nj])
        alphamax = (k+1)/min([Ni, Nj]) # where k = 1, 2, 3, ..., min(N-1)

        wavelengtharrmax[k-1] = 2*dx/alphamin
        wavelengtharrmin[k-1] = 2*dx/alphamax
        #wavelengtharrmid[k-1] = (wavelengtharrmin[k] + wavelengtharrmax[k])/2

        alpha_m = np.sqrt((Ni_inds**2/Ni**2) + (Nj_inds**2/Nj**2))

        a, b = weights(alphamin, alphamax, alpha_m)

        #vararr[k] = np.sum(totvar,  where = (alpha_m >= alphamin) & (alpha_m <= alphamax))
        vararr[k-1] += np.sum(a*totvar,  where = (alpha_m >= alphamin) & (alpha_m < alphamax))

        vararr[k] += np.sum(b*totvar,  where = (alpha_m >= alphamin) & (alpha_m < alphamax))

        if np.any(x < 0 for x in np.where((alpha_m >= alphamin) & (alpha_m < alphamax), a*totvar, 0)) == True:
            print('Negative values used, stop!')
            sys.exit()
        if np.any(x < 0 for x in np.where((alpha_m >= alphamin) & (alpha_m < alphamax), b*totvar, 0)) == True:
            print('Negative values used, stop!')
            sys.exit()
        # For each element in the variance array, compute alpha and add its contribution to the variance if alphamin <= alpha <= alphamax:
        ##for m in range(Ni):
         ##   for n in range(Nj):
                ##alpha = np.sqrt((m**2/Ni**2) + (n**2/Nj**2))
         ##       if alpha >= alphamin and alpha <= alphamax:
         ##           vararr[k] += totvar[n, m]
    return(wavelengtharrmax, vararr)

def weights(alpha_k, alpha_kp1, alpha_mn):
    """ From Ricard et al. (2013)"""
    a = (alpha_mn - alpha_k)/(alpha_kp1 - alpha_k)
    b = (alpha_kp1 - alpha_mn)/(alpha_kp1 - alpha_k)
    return(a, b)

# Make figure power spectrum

In [43]:
def make_power_spectrum(spatial_frequencies_1, amplitude_bins_1, sizefont, sizefont_legend, lead_times, map_params, date_task, paths, saving = False):
    colorscale = plt.cm.gnuplot
    colors = colorscale(np.linspace(0, 1, 11))
    labels_fig = ["a)", "b)", "c)", "d)"]
    #
    plt.rc("xtick", labelsize = sizefont)
    plt.rc("ytick", labelsize = sizefont)
    fig, ax = plt.subplots(3, 2, figsize = (22, 15), facecolor = "w", edgecolor = "k")

    for lti, leadtime in enumerate(lead_times):
        print(lti, leadtime)
        date_lt = (datetime.datetime.strptime(date_task, "%Y%m%d") + datetime.timedelta(days = int(leadtime))).strftime("%Y%m%d")
        ax = plt.subplot(3, 2, lti * 2 + 1, projection = map_params["map_proj"])
        ocean_feature = cartopy.feature.NaturalEarthFeature("physical", "ocean", "50m", edgecolor = "face", facecolor = "black")
        ax.add_feature(ocean_feature, zorder = 0)
        ax.set_extent(map_params["map_extent"], crs = cartopy.crs.PlateCarree())
        ax.add_feature(map_params["LAND_highres"], zorder = 1)
        cs = ax.pcolormesh(Dataset["lon"], Dataset["lat"], Dataset["SIC"][leadtime,0:-1,0:-1], transform = ccrs.PlateCarree(), norm = map_params["norm"], cmap = map_params["colormap"], zorder = 0, shading = "flat")
        if leadtime == 0:
            ax.set_title("MET-AICE " + date_lt[6:8] + "-" + date_lt[4:6] + "-" + date_lt[0:4] + "\n lead time: " + str(leadtime + 1) + " day", fontsize = sizefont)
        else:
            ax.set_title("MET-AICE " + date_lt[6:8] + "-" + date_lt[4:6] + "-" + date_lt[0:4] + "\n lead time: " + str(leadtime + 1) + " days", fontsize = sizefont)
        ax.text(-0.1, 0, labels_fig[lti], fontsize = sizefont, ha = "left", transform = ax.transAxes) 

    ax = plt.subplot(3,2,(2,6))
    for lt in lead_times:
        if lt == 0:
            l = ax.plot(spatial_frequencies_1[str(lt)], amplitude_bins_1[str(lt)], color = colors[lt], marker = ".", linewidth = 1.5, label = "Lead time: " + str(lt + 1) + " day", alpha = 1)
        else:
            l = ax.plot(spatial_frequencies_1[str(lt)], amplitude_bins_1[str(lt)], color = colors[lt], marker = ".", linewidth = 1.5, label = "Lead time: " +str(lt + 1) + " days", alpha = 1)
    ax.invert_xaxis()
    ax.set_xlabel("Wavelength (m)", fontsize = sizefont)
    ax.set_ylabel("Power", fontsize = sizefont)
    ax.set_title("Power Spectrum", fontsize = sizefont)
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.legend(fontsize = sizefont_legend)
    ax.text(-0.1, 0, labels_fig[3], fontsize = sizefont, ha = "left", transform = ax.transAxes) 
    #
    cbar_ax_1 = fig.add_axes([0.415, 0.16, 0.03, 0.7])
    cbar_1 = fig.colorbar(cs, cax = cbar_ax_1, ticks = map_params["levels"], orientation = "vertical", extend = None)
    cbar_1.set_label("Sea ice concentration (%)", fontsize = sizefont)
    
    if saving == True:
        plt.savefig(paths["output"] + "Power_spectrum_AICE_" + date_task + ".png", bbox_inches = "tight", dpi = 300)
    else:
        plt.show()

# Data processing

In [ ]:
Dataset = read_AICE(date_task, paths, xmin, xmax, ymin, ymax)
wavelengtharrmax_1 = {}
vararr_1 = {}
for lt in range(0, 10):
    wavelengtharrmax_1[str(lt)], vararr_1[str(lt)] = spectral_variance(Dataset["SIC"][lt,:,:], spatial_resolution)

make_power_spectrum(wavelengtharrmax_1, vararr_1, sizefont = sizefont, sizefont_legend = sizefont_legend, lead_times = lead_times, map_params = map_params, date_task = date_task, paths = paths, saving = True)

0 0
1 4
2 9


/tmp/ipykernel_357334/4165125157.py:13: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  ax = plt.subplot(3, 2, lti * 2 + 1, projection = map_params["map_proj"])
/tmp/ipykernel_357334/4165125157.py:13: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  ax = plt.subplot(3, 2, lti * 2 + 1, projection = map_params["map_proj"])
/tmp/ipykernel_357334/4165125157.py:13: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  ax = plt.subplot(3, 2, lti * 2 + 1, projection = map_params["map_proj"])
/tmp/ipykernel_357334/4165125157.py:25: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be 